In [2]:
import torch, onnx

m = '../data/conv.onnx'
onnx_model = onnx.load(m)
print(onnx_model)

ir_version: 7
producer_name: "pytorch"
producer_version: "1.10"
graph {
  node {
    input: "input"
    input: "conv1.weight"
    input: "conv1.bias"
    output: "5"
    name: "Conv_0"
    op_type: "Conv"
    attribute {
      name: "dilations"
      ints: 1
      ints: 1
      type: INTS
    }
    attribute {
      name: "group"
      i: 1
      type: INT
    }
    attribute {
      name: "kernel_shape"
      ints: 3
      ints: 3
      type: INTS
    }
    attribute {
      name: "pads"
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "strides"
      ints: 1
      ints: 1
      type: INTS
    }
  }
  node {
    input: "input"
    input: "conv2.weight"
    input: "conv2.bias"
    output: "6"
    name: "Conv_1"
    op_type: "Conv"
    attribute {
      name: "dilations"
      ints: 1
      ints: 1
      type: INTS
    }
    attribute {
      name: "group"
      i: 1
      type: INT
    }
    attribute {
      name: "kernel_shape

### initializer
initializer存储了模型中所有op的参数，例如conv.weight:

In [10]:
print(onnx_model.graph.initializer[0])

for i in onnx_model.graph.initializer:
    print(i.name, i.dims, i.data_type)

dims: 3
dims: 3
dims: 3
dims: 3
data_type: 1
name: "conv1.weight"
raw_data: "\001\202,\274\260h\032>b\234\033>@\316\234\2746\267\306\275\256\355\201\274\364\241\r\276\223\344\023\276R\245\236<\325\230\014\275\034\252f\275\300{\205\275)\377\t\276\362\361\244\275\322\261\220\275R>\n\276\304\037\374=W2\312\273{\'/>G\022\247=\237\236\031=\314w\350\275&\375\367=\360\374\357=\232K\325=6\277Q=/y\301\275\234n\004\276\344@\306\275\376a/>\361\255\021<3k\026>\037\177\000>J4,\276\t\222\033\276\010\303\220\275aeW=\256W\267\275\177S\010>\037\346\317\274*\031\334\275\003bT=^+\'\275\025s\222\275]\336%\276M\\\021\276\356\033\025>NB%\273\177\243\337=V\221\250\275|3\320\274\363&!>\375\253\366\275\3337\242\275\n\300o\275\020S3>5\004\344\275-$\362;a\360\020\272j!\230\275]\250\034\276\214\254:\2768\237J=E\207\314==\211$=z\315\352\273?(\330\275\234\034\017>q\007)\275B\004!>\013=!\276\270\203j\275\226\322\200\274\256\2158=\302Y\276<\236Lv\275M\201\366=\251\226B>\240\375\004\2752\023\226;\211\225\022>"

conv1.

In [12]:
class GraphParser:
    def __init__(self, model:onnx.ModelProto):
        self.m = model
        self.initializer_names = [x.name for x in model.graph.initializer]

    def _get_args(self, node: onnx.ModelProto):
        return [ipt for ipt in node.input if ipt in self.initializer_names]

    def _get_innodes(self, node: onnx.NodeProto):
        return [ipt for ipt in node.input if ipt not in self.initializer_names]

    def _get_attr_names(self, node: onnx.NodeProto):
        return [attr.name for attr in node.attribute]

    def parse(self):
        nodes = self.m.graph.node
        for node in nodes:
            print(node.op_type, node.name, self._get_innodes(node), 
                  self._get_args(node), self._get_attr_names(node), node.output)

GraphParser(onnx_model).parse()

Conv Conv_0 ['input'] ['conv1.weight', 'conv1.bias'] ['dilations', 'group', 'kernel_shape', 'pads', 'strides'] ['5']
Conv Conv_1 ['input'] ['conv2.weight', 'conv2.bias'] ['dilations', 'group', 'kernel_shape', 'pads', 'strides'] ['6']
Add Add_2 ['5', '6'] [] [] ['7']


### input

In [21]:
for i in onnx_model.graph.input:
    print(i.name, ':', i.type)
    # print(i.type)

input : tensor_type {
  elem_type: 1
  shape {
    dim {
      dim_value: 1
    }
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 20
    }
    dim {
      dim_value: 20
    }
  }
}

